In [ ]:

key ="2201050767bb417f928e06c7166304c6b41e485d"
b_key ="641DEDB0-361B-441D-BF2E-215378D97EB4"
from census import Census
from us import states
%matplotlib inline
import seaborn as sns


import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
pd.options.mode.chained_assignment = None 
import os
import requests
import urllib.request, json 
import pybea


c = Census(key)


In [3]:

data = pybea.get_data(b_key,
                      DataSetName='RegionalIncome',
                      TableName='CA1',
                      LineCode=3,
                      GeoFips='COUNTY',
                      Year=[ '2016'],
                      ResultFormat='JSON')
data = pd.DataFrame(data)
 


In [4]:
data.head(10)

,CL_UNIT,Code,DataValue,GeoFips,GeoName,NoteRef,TimePeriod,UNIT_MULT
0,dollars,CA1-3,49246.0,00000,United States,NaN,2016,0
1,dollars,CA1-3,38896.0,01000,Alabama,NaN,2016,0
2,dollars,CA1-3,39721.0,01001,"Autauga, AL",NaN,2016,0
3,dollars,CA1-3,41286.0,01003,"Baldwin, AL",NaN,2016,0
4,dollars,CA1-3,31788.0,01005,"Barbour, AL",NaN,2016,0
5,dollars,CA1-3,29264.0,01007,"Bibb, AL",NaN,2016,0
6,dollars,CA1-3,31470.0,01009,"Blount, AL",NaN,2016,0
7,dollars,CA1-3,26661.0,01011,"Bullock, AL",NaN,2016,0
8,dollars,CA1-3,33694.0,01013,"Butler, AL",NaN,2016,0
9,dollars,CA1-3,34401.0,01015,"Calhoun, AL",NaN,2016,0


In [ ]:
data2.head(10)

In [ ]:
df = pd.read_excel("County_Rural_Lookup.xlsx")
df = df.drop("Unnamed: 8", axis =1)


In [ ]:
df.columns = df.iloc[2]
df = df.drop([0, 1,2])
df = df.drop([3145, 3146,3147,3148,3149,3150])
df = df.rename(columns ={"2010 Census \nPercent Rural":"percent_rural"})
df = df.rename(columns = {"2015 GEOID": "geo_id"} )


In [ ]:
rural = df[df.percent_rural >= 99.9]

len(rural)

In [ ]:
m_rural = df[(df.percent_rural < 99.9) & (df.percent_rural > 49.9 )]
len(m_rural)

In [ ]:
urban = df[df.percent_rural <= 49.9]
len(urban)

In [ ]:
areas = pd.read_csv("final_county.csv")


In [ ]:
areas = areas.drop(["Unnamed: 0" ,"B01001_003E","B01001_004E","B01001_005E","B01001_006E"
                   ,"B01001_027E","B01001_028E","B01001_029E","B01001_030E"],axis = 1)


In [ ]:
areas.County = areas.County.astype(str)


In [ ]:
areas['County'] = areas['County'].str.zfill(3)

In [ ]:
areas.State = areas.State.astype(str)
areas['State'] = areas['State'].str.zfill(2)


In [ ]:
areas["geo_id"] = areas.State + areas.County


In [ ]:
unemployment = pd.read_excel("laucntycur14.xlsx")
unemployment.columns = unemployment.iloc[3]

unemployment = unemployment.drop([0, 1,2,3,4])

unemployment = unemployment.rename(index=str,columns ={"(%)":"unemployment_rate"})
unemployment["geo_id"] = unemployment["S_Code"] + unemployment["Code"]
unemployment.head()


In [ ]:
x = rural.loc[:, ['geo_id']]
areas.head()

In [ ]:
rural_result = pd.merge(x, areas, on='geo_id')

In [ ]:
rural_result = pd.merge(rural_result,unemployment, on ="geo_id")

rural_result = rural_result[rural_result["Period"] == "Jun-18"]

len(rural_result)
rural_result.to_excel("final_rural.xlsx")
#df3 = df1[(~df1.col1.isin(common.col1))

In [ ]:
data["geo_id"] = data["geo_id"].str.zfill(5)
rural_result = pd.merge(rural_result,data, on ="geo_id")
rural_result.to_excel("bea_final_rural.xlsx")

In [ ]:
len(rural_result)

In [ ]:
x = urban.loc[:, ['geo_id']]
urban_result = pd.merge(x, areas, on='geo_id')
len(urban_result)

In [ ]:
urban_result = pd.merge(urban_result,unemployment, on ="geo_id")


urban_result = urban_result[urban_result["Period"] == "Jun-18"]
len(urban_result)
urban_result.to_excel("final_urban.xlsx")

In [ ]:
d = pd.read_excel("test_d_final.xlsx")
d["geo_id"] = d["geo_id"].astype(str)
d["geo_id"] = d["geo_id"].str.zfill(5)

test_urban = pd.merge(urban_result,d, on ="geo_id")
test_urban.to_excel("beafinal_urban.xlsx")
len(test_urban)

In [ ]:
x = m_rural.loc[:, ['geo_id']]
m_rural_result = pd.merge(x, areas, on='geo_id')


In [ ]:
m_rural_result = pd.merge(m_rural_result,unemployment, on ="geo_id")

m_rural_result = m_rural_result[m_rural_result["Period"] == "Jun-18"]
m_rural_result.to_excel("final_mrural.xlsx")
testm_rural = pd.merge(m_rural, d, on ="geo_id")
testm_rural.to_excel("beafinal_mrural.xlsx")
#y = pd.merge(m_rural_result, testm_rural, on ="geo_id", how = "outer",indicator = True)
